## WPL Financial Model

#### Notes
1) Need to figure out how to add solar extension period + end effects period
2) Move functions into separate .py file after finishing
3) Dictionary of dfs of depreciations?
4) start thinking about how to automate QC'ing - ask Barett and Rowen about what they do to QC typically
5) Create functions to apply formatting
    - https://pandas.pydata.org/docs/user_guide/style.html
    - also from openpyxl.styles import Font, PatternFill, Alignment
6) automate making tables in "Financial Portfolios" tab --> the ones that are not hard coded

### Import packages

In [2]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import datetime as dt
from datetime import datetime
import pathlib

# Set the path to the folder containing your Excel files
folder_path = '/Users/alomsadze/OneDrive - Charles River Associates International/Desktop/WPL/Python Version/'
model_inputs = pd.ExcelFile(folder_path + "Direct Model Inputs.xlsx")
model_inputs

# Pause future warnings for cleaner output
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

### Supporting functions -- move to separate python file later

In [3]:
def read_excel_with_tables(df):
    """
    Read a single Excel sheet with multiple tables separated by an empty row.
    Each table is identified by the word "table" in a cell, and its name is in the next cell.

    Parameters:
    - file_path: str, path to the Excel file
    - sheet_name: str, name of the sheet in the Excel file

    Returns:
    - Dictionary of DataFrames, where keys are table names and values are corresponding DataFrames
    """

    # Initialize variables
    tables = {}
    current_table_name = None
    current_table_start_row = None

    # Iterate through rows
    for index, row in df.iterrows():
        if "table" in str(row.values).lower():
            # Save the current table if it exists
            if current_table_name is not None:
                data_table = df.iloc[current_table_start_row:index - 1, :].reset_index(drop=True)
                # Set the column names to be the values in the first row
                data_table.columns = data_table.iloc[0]
                # Drop the first row, which is now redundant as column headers
                data_table = data_table[1:]
                # Drop columns without names
                data_table = data_table.dropna(axis=1, how='all')
                # Now df contains your data with only columns that have column names
                tables[current_table_name] = data_table

            # Update variables for the new table
            current_table_name = str(row[1]) # this takes the value from the cell next to the cell that says "table"
            #####print("current table name is", current_table_name)
            current_table_start_row = index + 1  # Assuming the next row is empty
            
    # Add the last table
    if current_table_name is not None:
        data_table = df.iloc[current_table_start_row:index + 1, :].reset_index(drop=True)
        # Set the column names to be the values in the first row
        data_table.columns = data_table.iloc[0]
        # Drop the first row, which is now redundant as column headers
        data_table = data_table[1:]
        # Drop columns without names
        data_table = data_table.dropna(axis=1, how='all')
        # Now df contains your data with only columns that have column names
        tables[current_table_name] = data_table
        
    return tables

def get_resource_FOM(name_of_resource):
    
    resource_capacity_yearly = cumulative_installed_capacity_MW_df[name_of_resource]
    resource_capacity_yearly = resource_capacity_yearly * 1000 # convert MWs to KWs
    resource_FOM_yearly = FOM_2021_kw_year_df[name_of_resource]
    # Merge DataFrames on the 'Year' column
    resource_yearly_df = pd.merge(resource_capacity_yearly, resource_FOM_yearly, on='Year')
    resource_yearly_df = pd.merge(resource_yearly_df, inflation_vector, on='Year')
    resource_yearly_df.columns = ['Capacity','FOM', 'Inflation Scalar']
    # Multiply corresponding values for total FOM
    resource_yearly_df['Total FOM'] =  resource_yearly_df['Capacity'] * resource_yearly_df['FOM'] * resource_yearly_df['Inflation Scalar']
    resource_total_FOM_yearly = resource_yearly_df['Total FOM'] 
    resource_total_FOM_yearly = resource_total_FOM_yearly.to_frame().T

    return(resource_total_FOM_yearly)


### Import Data

In [4]:
financial_inputs = model_inputs.parse("Financial Inputs", header=None)
financial_inputs_tables = read_excel_with_tables(financial_inputs)
financial_scalars_inputs = financial_inputs_tables['Scalar Inputs'].set_index('Scalar Input')
inflation_vector_df = financial_inputs_tables['Inflation Vector - Base Year 2021$']
inflation_vector = inflation_vector_df.set_index('Year')['Scalar']
CCS_inputs = model_inputs.parse("CCS", header=None)
CCS_inputs_tables = read_excel_with_tables(CCS_inputs)

In [5]:
aurora_portfolio_summary = model_inputs.parse("Portfolio Summary")
capacity_payments = model_inputs.parse("Capacity Payments")
datacenter_scenario_financials = model_inputs.parse("Datacenter Scenario Financials", header=None)
hydrogen_island_inputs = read_excel_with_tables(model_inputs.parse("Hydrogen Island", header=None))

## Revenue Requirement Calculations

In [6]:
# GLOBAL VARIABLES TO GENERALIZE
Run_ID = 'CIC'
Scenario = 'Continue_Change'
year = 2023 
aurora_portfolio_ID = 2
aurora_condition = 'ATC'
case_name = "Datacenter"

In [7]:
# years we want to calculate revenue requirement for
start_year = 2023
end_year = 2072
years = np.arange(start_year, end_year + 1)

### 1. O&M Summary

#### 1a. Variable O&M

In [8]:
# Total owned resources costs (no market purchases/contracts)
total_owned_cost_yearly = []
# Market purchases cost
market_purchases_yearly = []
# Revenues from market sales
market_sales_yearly = []
# Cost of contracts
contract_costs_yearly = []
# Contract Sales
contract_sales_yearly = []
# High Load Capacity Payments = Cost of short capacity in high load scenario
capacity_payments_yearly = []

condition_mask = aurora_portfolio_summary['Condition'] == aurora_condition
run_id_mask = aurora_portfolio_summary['Run_ID'] == Run_ID
portfolio_id_mask = aurora_portfolio_summary['Portfolio_ID'] == aurora_portfolio_ID
aurora_portfolio_summary_filtered = aurora_portfolio_summary[condition_mask & run_id_mask & portfolio_id_mask]

capacity_payments_filtered = capacity_payments[(capacity_payments.Scenarios == Scenario) & (capacity_payments['Case Name'] == case_name)]

aurora_years = np.sort(aurora_portfolio_summary.Time_Period.unique())

for year in aurora_years:
    # select row for current year
    curr_year_data = aurora_portfolio_summary_filtered.loc[(aurora_portfolio_summary_filtered['Time_Period'] == year)]
    curr_year_capacity_payments = capacity_payments_filtered[year]
    # extract data
    total_owned_cost_yearly.append(curr_year_data["Resource_Cost_Total"].sum() * 1000)
    market_purchases_yearly.append(curr_year_data['Market_Purchases_Cost_Total'].sum() * 1000)
    market_sales_yearly.append(curr_year_data['Market_Sales_Cost_Total'].sum() * 1000)
    contract_costs_yearly.append(curr_year_data['Contract_Purchases_Cost_Total'].sum() * 1000)
    contract_sales_yearly.append(curr_year_data['Contract_Sales_Cost_Total'].sum() * 1000)
    capacity_payments_yearly.append(curr_year_capacity_payments.sum())
    
# Summarize
# Net market purchases = Purchases + Sales
net_market_purchases_yearly = [sum(costs) for costs in zip(market_purchases_yearly, market_sales_yearly)]
total_portfolio_cost_yearly= [sum(costs) for costs in zip(total_owned_cost_yearly, net_market_purchases_yearly,
                                                  contract_costs_yearly, contract_sales_yearly)]
total_portfolio_cost_yearly

# dictionary of lists 
VOM_portfolio_cost_dict = {'Year': aurora_years,
        'Total Portfolio Cost': total_portfolio_cost_yearly, 
        'Total Owned Cost': total_owned_cost_yearly, 
        'Net Market Purchases': net_market_purchases_yearly, 
        'Market Purchases (Energy)': market_purchases_yearly,
        'Market Sales (Energy)': market_sales_yearly, 
        'Contract Cost': contract_costs_yearly, 
        'Contract Sales': contract_sales_yearly,
        'High Load Capacity Payment': capacity_payments_yearly} 
   
VOM_portfolio_cost_df = pd.DataFrame(VOM_portfolio_cost_dict)
VOM_portfolio_cost_df.set_index('Year', inplace=True)
VOM_portfolio_cost_df = VOM_portfolio_cost_df.T
VOM_portfolio_cost_df.style.format(precision=2)

Year,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040,2041,2042,2043,2044,2045,2046
Total Portfolio Cost,360114373.05,315543248.05,254601854.00,266444571.78,297370895.02,316452912.11,335602806.27,340641207.03,325180210.94,329969429.69,320359316.41,334487855.47,251862718.75,256107863.28,264522406.25,278117394.53,294598144.53,319782792.97,339273960.94,357824754.88,378143000.00,392755512.45,333651893.31,341379343.26,267445956.30
Total Owned Cost,537460187.50,471145218.75,353665406.25,262398250.00,253670625.00,269277125.00,293761968.75,308243187.50,325586125.00,245003406.25,229553406.25,228581312.50,267606250.00,267590437.50,263890875.00,268308562.50,276112906.25,270473062.50,286528218.75,293996718.75,333906562.50,333948343.75,350943906.25,232417156.25,234832531.25
Net Market Purchases,-211586578.12,-206615415.04,-155103590.82,-45490605.47,-1956746.09,880042.97,-9773449.22,-21969156.25,-55561976.56,28918558.59,35088441.41,49422453.12,-66399035.16,-52659835.94,-41258140.62,-32482687.50,-24464847.66,5588796.88,11931453.12,51119648.44,31557671.88,54768781.25,-21341078.12,104923101.56,28574812.50
Market Purchases (Energy),27016125.00,16312913.09,15318784.18,37247597.66,57834707.03,64844214.84,63107496.09,53082539.06,48967757.81,86054789.06,90726929.69,100006851.56,42647191.41,49081226.56,54515609.38,60523109.38,64799777.34,82657304.69,91013796.88,120368687.50,116709039.06,133438296.88,130535000.00,197774578.12,175095375.00
Market Sales (Energy),-238602703.12,-222928328.12,-170422375.00,-82738203.12,-59791453.12,-63964171.88,-72880945.31,-75051695.31,-104529734.38,-57136230.47,-55638488.28,-50584398.44,-109046226.56,-101741062.50,-95773750.00,-93005796.88,-89264625.00,-77068507.81,-79082343.75,-69249039.06,-85151367.19,-78669515.62,-151876078.12,-92851476.56,-146520562.50
Contract Cost,39703292.97,56600320.31,61741785.16,55335277.34,51587679.69,52316910.16,53354585.94,54367175.78,55156062.50,56047464.84,55717468.75,56484089.84,50655503.91,41177261.72,41889671.88,42291519.53,42950085.94,43720933.59,40814289.06,12708387.70,12678765.62,4038387.45,4049065.19,4039085.45,4038612.55
Contract Sales,-5462529.30,-5586875.98,-5701746.58,-5798350.10,-5930663.57,-6021166.02,-1740299.19,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
High Load Capacity Payment,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


#### 1b. Other Fixed Costs

##### 1b.i FOM, Tax, and New Unit FOM

In [9]:
datacenter_scenario_financials_tables = read_excel_with_tables(datacenter_scenario_financials)
list(datacenter_scenario_financials_tables.keys())

['Ongoing CapEx (Nominal$)',
 'FOM (Nominal$)',
 'Decomissioning',
 'Transmission Upgrade Costs - DR Costs',
 'Existing Capital -  Net Book Value BOY',
 'Existing Capital -  Net Book Value EOY',
 'Existing Capital - Tax Value - Federal - BOY',
 'Existing Capital - Tax Value - Federal - EOY',
 'Existing Capital - Tax Value - State - BOY',
 'Existing Capital - Tax Value - State - EOY',
 'Retired',
 'Distribution Offset',
 'Tax Equity Costs',
 'Cumulative Installed Capacity (MW)',
 'Ongoing CapEx ($2021/kW-yr)',
 'Fixed O&M ($2021/kW-yr)',
 'Distribution Offset ($2021/kW)',
 'RT Subhourly and A/S Offset ($2021/kW-yr)']

In [44]:
# FORMAT SOME INPUT TABLES FOR EASIER DATA ACCESS
cumulative_installed_capacity_MW_df = datacenter_scenario_financials_tables['Cumulative Installed Capacity (MW)'].T
cumulative_installed_capacity_MW_df.columns = cumulative_installed_capacity_MW_df.iloc[0]
cumulative_installed_capacity_MW_df = cumulative_installed_capacity_MW_df[1:].rename_axis('Year')

FOM_2021_kw_year_df = datacenter_scenario_financials_tables['Fixed O&M ($2021/kW-yr)'].T
FOM_2021_kw_year_df.columns = FOM_2021_kw_year_df.iloc[0]
FOM_2021_kw_year_df = FOM_2021_kw_year_df[1:].rename_axis('Year')

FOM_years = cumulative_installed_capacity_MW_df.index.values

# MAKE ROWS FOR TABLE
transmission_upgrade_costs_df = datacenter_scenario_financials_tables['Transmission Upgrade Costs - DR Costs']
FOM_yearly = transmission_upgrade_costs_df[transmission_upgrade_costs_df['Category'] == 'FOM']
FOM_yearly = FOM_yearly.loc[:, FOM_yearly.columns.isin(FOM_years)].values[0]
Transmission_Upgrade_OpEx_yearly = transmission_upgrade_costs_df[transmission_upgrade_costs_df['Category'] == 'Transmission Upgrade OpEx']
Transmission_Upgrade_OpEx_yearly = Transmission_Upgrade_OpEx_yearly.loc[:, Transmission_Upgrade_OpEx_yearly.columns.isin(FOM_years)].values[0]
DSM_Costs_yearly = transmission_upgrade_costs_df[transmission_upgrade_costs_df['Category'] == 'DSM Costs']
DSM_Costs_yearly = DSM_Costs_yearly.loc[:, DSM_Costs_yearly.columns.isin(FOM_years)].values[0]
PTC_or_ITC = financial_scalars_inputs.loc['Long-term solar projects ITCs or PTCs?'].values[0]
if PTC_or_ITC == "ITC":
    tax_equity_costs_df = datacenter_scenario_financials_tables['Tax Equity Costs']
    tax_equity_costs_CA1_CA2_yearly = tax_equity_costs_df[tax_equity_costs_df.Category == 'Cash Distributions/OpEx for TE - CA1 & CA2']
    tax_equity_costs_CA1_CA2_yearly = tax_equity_costs_CA1_CA2_yearly.loc[:, tax_equity_costs_CA1_CA2_yearly.columns.isin(FOM_years)].values[0]
    tax_equity_costs_longterm_solar_yearly = tax_equity_costs_df[tax_equity_costs_df.Category == 'Cash Distributions/OpEx for TE- Long-Term Solar']
    tax_equity_costs_longterm_solar_yearly = tax_equity_costs_longterm_solar_yearly.loc[:, tax_equity_costs_longterm_solar_yearly.columns.isin(FOM_years)].values[0]
else:
    tax_equity_costs_CA1_CA2_yearly = [0] * len(FOM_years)
    tax_equity_costs_longterm_solar_yearly = [0] * len(FOM_years)

wind_FOM_yearly = get_resource_FOM('Wind - WI').values[0]
H2_Enabled_RICE_FOM_yearly = get_resource_FOM('H2 Enabled RICE').values[0]
Paired_Li_Ion_Battery_10_Bonus_ITC_FOM_yearly = get_resource_FOM('Paired Li Ion Battery_10% Bonus ITC').values[0]
Standalone_Li_Ion_Battery_FOM_yearly = get_resource_FOM('Standalone Li Ion Battery').values[0]
Paired_Li_Ion_Battery_FOM_yearly = get_resource_FOM('Paired Li Ion Battery').values[0]
Paired_Li_Ion_Battery_no_ITC_FOM_yearly = get_resource_FOM('Paired Li Ion Battery_no ITC').values[0]
Solar_and_Storage_WI_FOM_yearly = get_resource_FOM('Solar and Storage - WI').values[0]
CA1_2022_FOM_yearly = get_resource_FOM('CA1 - 2022').values[0]
CA1_2023_FOM_yearly = get_resource_FOM('CA1 - 2023').values[0]
CA2_2023_FOM_yearly = get_resource_FOM('CA2 - 2023').values[0]
Long_Term_Solar_FOM_yearly = get_resource_FOM('Long-Term Solar').values[0]
Flow_Battery_Med_8_hrs_FOM_yearly = get_resource_FOM('Flow Battery - Med 8 hrs').values[0]
AGP_Neenah_Sheboygan_FOM_yearly = get_resource_FOM('AGP Neenah & Sheboygan').values[0]
AGP_Riverside_FOM_yearly = get_resource_FOM('AGP Riverside').values[0]

# Gas CCGT FOM is a bit more involved because we need to add in separate CSS FOM values
# NOTE - CSS FOM starts a year before we actually have capacity. TBD on what to do here.
# This is also currently incorrectly coded in the Excel model - CSS values offset by a year in the summation.
Gas_CCGT_with_CCS_FOM_yearly = get_resource_FOM('Gas CCGT with CCS')
CCS_FOM_df = CCS_inputs_tables['$ FOM'][CCS_inputs_tables['$ FOM']['Aurora ID'] == aurora_portfolio_ID].drop(columns='Aurora ID')
Gas_CCGT_FOM_df = pd.concat([CCS_FOM_df, Gas_CCGT_with_CCS_FOM_yearly])
Gas_CCGT_with_CCS_FOM_yearly = Gas_CCGT_FOM_df.sum(skipna=True).values

# Gas CCGT FOM is a bit more involved because we need to use a separate input that is nominal (not pre-inflated)
hydrogen_island_capacity_yearly = cumulative_installed_capacity_MW_df['H2 Island']
hydrogen_island_capacity_yearly = hydrogen_island_capacity_yearly * 1000 # convert MWs to KWs
hydrogen_island_FOM = hydrogen_island_inputs['FOM'].set_index('Year')
hydrogen_island_FOM_yearly = hydrogen_island_FOM_df[Scenario]
hydrogen_island_yearly_df = pd.merge(hydrogen_island_capacity_yearly, hydrogen_island_FOM_yearly, on='Year')
hydrogen_island_yearly_df.columns = ['Capacity','FOM']
hydrogen_island_yearly_df['Total FOM'] =  hydrogen_island_yearly_df['Capacity'] * hydrogen_island_yearly_df['FOM']
hydrogen_total_FOM_yearly = hydrogen_island_yearly_df['Total FOM'] 
H2_island_FOM_yearly = hydrogen_total_FOM_yearly.to_frame().T
H2_island_FOM_yearly = H2_island_FOM_yearly.values[0]

new_unit_FOM_yearly = np.sum([wind_FOM_yearly,
                           H2_island_FOM_yearly,
                           H2_Enabled_RICE_FOM_yearly,
                           Paired_Li_Ion_Battery_10_Bonus_ITC_FOM_yearly,
                           Standalone_Li_Ion_Battery_FOM_yearly,
                           Paired_Li_Ion_Battery_FOM_yearly,
                           Paired_Li_Ion_Battery_no_ITC_FOM_yearly,
                           Solar_and_Storage_WI_FOM_yearly,
                           CA1_2022_FOM_yearly,
                           CA1_2023_FOM_yearly,
                           CA2_2023_FOM_yearly,
                           Long_Term_Solar_FOM_yearly,
                           Flow_Battery_Med_8_hrs_FOM_yearly,
                           Gas_CCGT_with_CCS_FOM_yearly,
                           AGP_Neenah_Sheboygan_FOM_yearly,
                           AGP_Riverside_FOM_yearly], axis=0)

# dictionary of lists 
FOM_portfolio_cost_dict = {'Year': FOM_years,
                           'FOM': FOM_yearly,
                           'Transmission Upgrade OpEx': Transmission_Upgrade_OpEx_yearly,
                           'DSM Costs': DSM_Costs_yearly,
                           'Tax Equity Costs - CA1 & CA2': tax_equity_costs_CA1_CA2_yearly,
                           'Tax Equity Costs - Long-Term Solar': tax_equity_costs_longterm_solar_yearly,
                           'New Unit FOM': new_unit_FOM_yearly,
                           'Wind - WI - FOM': wind_FOM_yearly,
                           'H2 Island': H2_island_FOM_yearly,
                           'H2 Enabled RICE': H2_Enabled_RICE_FOM_yearly,
                           'Paired Li Ion Battery_10% Bonus ITC': Paired_Li_Ion_Battery_10_Bonus_ITC_FOM_yearly,
                           'Standalone Li Ion Battery': Standalone_Li_Ion_Battery_FOM_yearly,
                           'Paired Li Ion Battery': Paired_Li_Ion_Battery_FOM_yearly,
                           'Paired Li Ion Battery_no ITC': Paired_Li_Ion_Battery_no_ITC_FOM_yearly,
                           'Solar and Storage - WI': Solar_and_Storage_WI_FOM_yearly,
                           'CA1 - 2022': CA1_2022_FOM_yearly,
                           'CA1 - 2023': CA1_2023_FOM_yearly,
                           'CA2 - 2023': CA2_2023_FOM_yearly,
                           'Long-Term Solar': Long_Term_Solar_FOM_yearly,
                           'Flow Battery - Med 8 hrs': Flow_Battery_Med_8_hrs_FOM_yearly,
                           'Gas CCGT with CCS': Gas_CCGT_with_CCS_FOM_yearly,
                           'AGP Neenah & Sheboygan': AGP_Neenah_Sheboygan_FOM_yearly,
                           'AGP Riverside': AGP_Riverside_FOM_yearly} 

FOM_portfolio_cost_df = pd.DataFrame(FOM_portfolio_cost_dict)
FOM_portfolio_cost_df.set_index('Year', inplace=True)
FOM_portfolio_cost_df = FOM_portfolio_cost_df.T
FOM_portfolio_cost_df.style.format(precision=0)

Year,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
FOM,50201215,59707422,56192857,58415754,71870574,68329971,70350836,62381446,71355625,43302290,44179983,45311958,45778741,47505085,48420670,48822430,51160695,51096582,51041855,51720035,52394896,52576138,52330266,33675491,34349001,26586154
Transmission Upgrade OpEx,0,122425,677965,677965,775528,3763883,5797957,7874746,9995147,12160077,14370471,14370471,15522573,15522573,15522573,15522573,15522573,15522573,16501402,18100418,20821408,20821408,37308402,37308402,40265257,40265257
DSM Costs,698000,923260,985775,1110848,1235920,1360993,1486065,1611138,1736210,1772671,1809897,1847905,1886711,1926332,1966785,2008087,2050257,2093312,2137272,2182155,2227980,2274768,2322538,2371311,2421108,2471952
Tax Equity Costs - CA1 & CA2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Tax Equity Costs - Long-Term Solar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
New Unit FOM,2680048,9258611,10356684,13713089,16020028,24436842,31434137,39414737,54099052,62166483,70564552,73592976,192009804,196415460,200570351,204208622,208081522,211274716,217192626,224414212,238491218,243984857,290596511,303714502,338196233,222103476
Wind - WI - FOM,0,0,0,0,0,6641556,13562057,20770290,28275289,36086337,44212980,45141453,46089423,47057301,48045504,49054460,50084604,51136380,52210244,53306660,63497116,64830555,75647997,77236605,88715895,90578929
H2 Island,0,0,0,0,0,0,0,0,0,0,0,0,0,640442,647765,655249,662898,670714,678700,686856,701280,716007,731043,746395,762070,778073
H2 Enabled RICE,0,0,0,0,0,1765704,1802784,2530883,7282270,7435198,7591337,7750755,7913521,8079705,8249379,8422615,8599490,9063308,10699518,11219457,11455066,12311181,18854574,19250520,19654781,20067531
Paired Li Ion Battery_10% Bonus ITC,0,0,0,3430823,3502870,3576430,3651535,3728218,3806510,3886447,3968062,4051392,4136471,4223337,4312027,4402579,4495034,4589429,4685807,4784209,4884678,4987256,5091988,5198920,5308097,5419567


##### 1b.ii New Unit Subhourly / Ancillary Revenue